In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from tqdm import tqdm_notebook
import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim 
import torchvision
from torchvision import models
from torch.utils.data import DataLoader, Dataset
import torch.utils.data as utils
from torchvision import transforms
import torch.nn.functional as F
import glob


In [2]:
file_path=glob.glob('./crack_train/*.png')
print(file_path)

['./crack_train/0.png', './crack_train/1.png', './crack_train/2.png', './crack_train/3.png', './crack_train/4.png', './crack_train/5.png', './crack_train/6.png', './crack_train/7.png', './crack_train/8.png', './crack_train/9.png', './crack_train/10.png', './crack_train/11.png', './crack_train/12.png', './crack_train/13.png', './crack_train/14.png', './crack_train/15.png', './crack_train/16.png', './crack_train/17.png', './crack_train/18.png', './crack_train/19.png', './crack_train/20.png', './crack_train/21.png', './crack_train/22.png', './crack_train/23.png', './crack_train/24.png', './crack_train/25.png', './crack_train/26.png', './crack_train/27.png', './crack_train/28.png', './crack_train/29.png', './crack_train/30.png', './crack_train/31.png', './crack_train/32.png', './crack_train/33.png', './crack_train/34.png', './crack_train/35.png', './crack_train/36.png', './crack_train/37.png', './crack_train/38.png', './crack_train/39.png', './crack_train/40.png', './crack_train/41.png', '

In [3]:
class ImageData(Dataset):
    def __init__(self,  transform, subset="train",damage_name='crack'):
        super().__init__()
        #self.df = df
        self.transform = transform
        self.subset = subset
        self.damage_name=damage_name
        self.fn=glob.glob('main_'+self.damage_name+'_'+self.subset+'/*.png')
        #if self.subset == "train":
        #    self.data_path = path + 'train_images/'
        #elif self.subset == "test":
        #    self.data_path = path + 'test_images/'

    def __len__(self):
        return len(self.fn)
    
    def __getitem__(self, index):  
        
        #fn = self.df['ImageId_ClassId'].iloc[index].split('_')[0]         
        img = Image.open( self.fn[index])
        img = self.transform(img)
        #print(img.shape)

        if self.subset == 'train': 
            #mask = rle2mask(self.df['EncodedPixels'].iloc[index], (256, 1600))
            mask = Image.open(self.fn[index].replace('main_',''))
            #mask = transforms.ToPILImage()(mask)            
            mask = self.transform(mask)
            return img, mask
        else: 
            mask = None
            return img       

In [4]:
def convrelu(in_channels, out_channels, kernel, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel, padding=padding),
        nn.ReLU(inplace=True),
    )

class UNet(nn.Module):
    def __init__(self, n_class):
        super().__init__()
        
        self.base_model = models.resnet18()
        self.base_model.load_state_dict(torch.load("pretrained/resnet18.pth"))
        self.base_layers = list(self.base_model.children())

        self.layer0 = nn.Sequential(*self.base_layers[:3])
        self.layer0_1x1 = convrelu(64, 64, 1, 0)
        self.layer1 = nn.Sequential(*self.base_layers[3:5])
        self.layer1_1x1 = convrelu(64, 64, 1, 0)
        self.layer2 = self.base_layers[5]
        self.layer2_1x1 = convrelu(128, 128, 1, 0)
        self.layer3 = self.base_layers[6]
        self.layer3_1x1 = convrelu(256, 256, 1, 0)
        self.layer4 = self.base_layers[7]
        self.layer4_1x1 = convrelu(512, 512, 1, 0)

        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.conv_up3 = convrelu(256 + 512, 512, 3, 1)
        self.conv_up2 = convrelu(128 + 512, 256, 3, 1)
        self.conv_up1 = convrelu(64 + 256, 256, 3, 1)
        self.conv_up0 = convrelu(64 + 256, 128, 3, 1)

        self.conv_original_size0 = convrelu(3, 64, 3, 1)
        self.conv_original_size1 = convrelu(64, 64, 3, 1)
        self.conv_original_size2 = convrelu(64 + 128, 64, 3, 1)

        self.conv_last = nn.Conv2d(64, n_class, 1)

    def forward(self, input):
        x_original = self.conv_original_size0(input)
        x_original = self.conv_original_size1(x_original)

        layer0 = self.layer0(input)
        layer1 = self.layer1(layer0)
        layer2 = self.layer2(layer1)
        layer3 = self.layer3(layer2)
        layer4 = self.layer4(layer3)

        layer4 = self.layer4_1x1(layer4)
        x = self.upsample(layer4)
        layer3 = self.layer3_1x1(layer3)
        x = torch.cat([x, layer3], dim=1)
        x = self.conv_up3(x)

        x = self.upsample(x)
        layer2 = self.layer2_1x1(layer2)
        x = torch.cat([x, layer2], dim=1)
        x = self.conv_up2(x)

        x = self.upsample(x)
        layer1 = self.layer1_1x1(layer1)
        x = torch.cat([x, layer1], dim=1)
        x = self.conv_up1(x)

        x = self.upsample(x)
        layer0 = self.layer0_1x1(layer0)
        x = torch.cat([x, layer0], dim=1)
        x = self.conv_up0(x)

        x = self.upsample(x)
        x = torch.cat([x, x_original], dim=1)
        x = self.conv_original_size2(x)

        out = self.conv_last(x)

        return out

In [5]:
# # Implementation from https://github.com/timctho/unet-pytorch/
# class UNet_down_block(torch.nn.Module):
#     def __init__(self, input_channel, output_channel, down_size):
#         super(UNet_down_block, self).__init__()
#         self.conv1 = torch.nn.Conv2d(input_channel, output_channel, 3, padding=1)
#         self.bn1 = torch.nn.BatchNorm2d(output_channel)
#         self.conv2 = torch.nn.Conv2d(output_channel, output_channel, 3, padding=1)
#         self.bn2 = torch.nn.BatchNorm2d(output_channel)
#         self.conv3 = torch.nn.Conv2d(output_channel, output_channel, 3, padding=1)
#         self.bn3 = torch.nn.BatchNorm2d(output_channel)
#         self.max_pool = torch.nn.MaxPool2d(2, 2)
#         self.relu = torch.nn.ReLU()
#         self.down_size = down_size

#     def forward(self, x):
#         if self.down_size:
#             x = self.max_pool(x)
#         x = self.relu(self.bn1(self.conv1(x)))
#         x = self.relu(self.bn2(self.conv2(x)))
#         x = self.relu(self.bn3(self.conv3(x)))
#         return x

# class UNet_up_block(torch.nn.Module):
#     def __init__(self, prev_channel, input_channel, output_channel):
#         super(UNet_up_block, self).__init__()
#         self.up_sampling = torch.nn.Upsample(scale_factor=2, mode='bilinear')
#         self.conv1 = torch.nn.Conv2d(prev_channel + input_channel, output_channel, 3, padding=1)
#         self.bn1 = torch.nn.BatchNorm2d(output_channel)
#         self.conv2 = torch.nn.Conv2d(output_channel, output_channel, 3, padding=1)
#         self.bn2 = torch.nn.BatchNorm2d(output_channel)
#         self.conv3 = torch.nn.Conv2d(output_channel, output_channel, 3, padding=1)
#         self.bn3 = torch.nn.BatchNorm2d(output_channel)
#         self.relu = torch.nn.ReLU()

#     def forward(self, prev_feature_map, x):
#         x = self.up_sampling(x)
#         x = torch.cat((x, prev_feature_map), dim=1)
#         x = self.relu(self.bn1(self.conv1(x)))
#         x = self.relu(self.bn2(self.conv2(x)))
#         x = self.relu(self.bn3(self.conv3(x)))
#         return x


# class UNet(torch.nn.Module):
#     def __init__(self):
#         super(UNet, self).__init__()

#         self.down_block1 = UNet_down_block(3, 16, False)
#         self.down_block2 = UNet_down_block(16, 32, True)
#         self.down_block3 = UNet_down_block(32, 64, True)
#         self.down_block4 = UNet_down_block(64, 128, True)
#         self.down_block5 = UNet_down_block(128, 256, True)
#         self.down_block6 = UNet_down_block(256, 512, True)
#         self.down_block7 = UNet_down_block(512, 1024, True)

#         self.mid_conv1 = torch.nn.Conv2d(1024, 1024, 3, padding=1)
#         self.bn1 = torch.nn.BatchNorm2d(1024)
#         self.mid_conv2 = torch.nn.Conv2d(1024, 1024, 3, padding=1)
#         self.bn2 = torch.nn.BatchNorm2d(1024)
#         self.mid_conv3 = torch.nn.Conv2d(1024, 1024, 3, padding=1)
#         self.bn3 = torch.nn.BatchNorm2d(1024)

#         self.up_block1 = UNet_up_block(512, 1024, 512)
#         self.up_block2 = UNet_up_block(256, 512, 256)
#         self.up_block3 = UNet_up_block(128, 256, 128)
#         self.up_block4 = UNet_up_block(64, 128, 64)
#         self.up_block5 = UNet_up_block(32, 64, 32)
#         self.up_block6 = UNet_up_block(16, 32, 16)

#         self.last_conv1 = torch.nn.Conv2d(16, 16, 3, padding=1)
#         self.last_bn = torch.nn.BatchNorm2d(16)
#         self.last_conv2 = torch.nn.Conv2d(16, 1, 1, padding=0)
#         self.relu = torch.nn.ReLU()

#     def forward(self, x):
#         self.x1 = self.down_block1(x)
#         self.x2 = self.down_block2(self.x1)
#         self.x3 = self.down_block3(self.x2)
#         self.x4 = self.down_block4(self.x3)
#         self.x5 = self.down_block5(self.x4)
#         self.x6 = self.down_block6(self.x5)
#         self.x7 = self.down_block7(self.x6)
#         self.x7 = self.relu(self.bn1(self.mid_conv1(self.x7)))
#         self.x7 = self.relu(self.bn2(self.mid_conv2(self.x7)))
#         self.x7 = self.relu(self.bn3(self.mid_conv3(self.x7)))
#         x = self.up_block1(self.x6, self.x7)
#         x = self.up_block2(self.x5, x)
#         x = self.up_block3(self.x4, x)
#         x = self.up_block4(self.x3, x)
#         x = self.up_block5(self.x2, x)
#         x = self.up_block6(self.x1, x)
#         x = self.relu(self.last_bn(self.last_conv1(x)))
#         x = self.last_conv2(x)
#         return x

In [6]:
data_transf = transforms.Compose([
                                  transforms.Scale((128, 128)),
                                  transforms.ToTensor()])
train_data = ImageData( transform = data_transf)
train_loader = DataLoader(dataset = train_data, batch_size=4)


/opt/conda/lib/python3.7/site-packages/torchvision/transforms/transforms.py:279: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


In [7]:
model = UNet(n_class=1).cuda()
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), weight_decay=1e-4, lr = 0.001, momentum=0.9)

In [ ]:
%%time
for epoch in range(100):      
    model.train()         
    for ii, (data, target) in enumerate(train_loader):                         
        data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)  
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()          
    print('Epoch: {} - Loss: {:.6f}'.format(epoch + 1, loss.item()))

Epoch: 1 - Loss: 0.698548
Epoch: 2 - Loss: 0.527437
Epoch: 3 - Loss: 0.522295
Epoch: 4 - Loss: 0.453132
Epoch: 5 - Loss: 0.393737
Epoch: 6 - Loss: 0.325637
Epoch: 7 - Loss: 0.275527
Epoch: 8 - Loss: 0.291598
Epoch: 9 - Loss: 0.262668
Epoch: 10 - Loss: 0.210288
Epoch: 11 - Loss: 0.211458
Epoch: 12 - Loss: 0.193895
Epoch: 13 - Loss: 0.160963
Epoch: 14 - Loss: 0.175982
Epoch: 15 - Loss: 0.145135
Epoch: 16 - Loss: 0.169036
Epoch: 17 - Loss: 0.241312
Epoch: 18 - Loss: 0.234839
Epoch: 19 - Loss: 0.213245
Epoch: 20 - Loss: 0.178859
Epoch: 21 - Loss: 0.132311
Epoch: 22 - Loss: 0.124248
Epoch: 23 - Loss: 0.120573
Epoch: 24 - Loss: 0.134003
Epoch: 25 - Loss: 0.126405
Epoch: 26 - Loss: 0.114323
Epoch: 27 - Loss: 0.108857
Epoch: 28 - Loss: 0.123341
Epoch: 29 - Loss: 0.131822
Epoch: 30 - Loss: 0.125046
Epoch: 31 - Loss: 0.105484
Epoch: 32 - Loss: 0.091703
Epoch: 33 - Loss: 0.090379
Epoch: 34 - Loss: 0.106057


In [ ]:
plt.imshow(train_data[19][0].permute(1, 2, 0))

In [ ]:
x = train_data[19][0].unsqueeze(0)
o = model(x.cuda())  
o = o.cpu().detach().numpy() * (-1)
tmp = np.copy(o)
mn = np.mean(o)*1.2
tmp[tmp<mn] = 0
tmp[tmp>mn] = 1
plt.imshow(np.squeeze(tmp))